[Reference](https://medium.com/wehkamp-techblog/implementing-dry-with-python-decorators-a5b7c68b6b64)

In [2]:
@app.route('/')
def index():
  return 'Welcome to my website!'

In [3]:
DATABASE_MODELS = []

def register(f):
  GLOBAL_ARRAY.append(f)
  return f


@register
class MyModel(DatabaseModel):
  pass


def setup_db():
  for model in DATABASE_MODEL:
    # Setup the models table

In [4]:
import functools
from datetime import datetime

from flask import request
from peewee import *
from playhouse.postgres_ext import *

class AuditModel(Model):
  id = AutoField()
  user = TextField()
  action = TextField()
  timestamp = DateTimeField()
  metadata = BinaryJSONField()
  
  abort = False
  
  class Meta:
    db_table = 'audits'

In [5]:
@staticmethod
def log(f=None):
  def deco(f):
    @functools.wraps(f)
    def func(*args, **kwargs):
      audit = AuditModel()
      user = request.headers.get("X-Auth-User")
      if user is None:
        return 'UNAUTHORIZED. Please provide X-Auth-User header.', 403
      audit.user = user
      audit.action = f'{f.__module__}.{f.__name__}'
      audit.timestamp = datetime.utcnow()
      if 'audit' in f.__code__.co_varnames:
        kwargs['audit'] = audit
      result = f(*args, **kwargs)
      if not audit.abort:
        audit.save(force_insert=True)
      return result
    return func

  if f and callable(f):
    return deco(f)

  return deco


In [6]:
@app.route('/item/<int:item>', methods=['DELETE'])
@AuditModel.log
def clear_all(item, audit):
  audit.metadata = { 'item_id': item }
  
  # DELETE item
  
  return '', 204

In [7]:
@staticmethod
def prefetch(f=None):
  def deco(f):
    @functools.wraps(f)
    def func(*args, **kwargs):
      try:
        item = Item.get(
          Item.id == kwargs.pop('item')
        )
        return f(item, *args, **kwargs)
      except Item.DoesNotExist:
        return 'Item not found', 404
    return func

  if f and callable(f):
    return deco(f)

  return deco

In [8]:
@app.route('/item/<int:item>', methods=['DELETE'])
@Item.prefetch
@AuditModel.log
def clear_all(item, audit):
  audit.metadata = { 'item_id': item.id }
  
  print(item) # Item instance. Not an int anymore
  # DELETE item
  
  return '', 204